In [16]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [17]:
def execute_trading_strategy(ticker_symbol, start_date, end_date):
    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

    ohlc_data = stock_data[['Open', 'High', 'Low', 'Close']]

    short_ema = ohlc_data['Close'].ewm(span=13, min_periods=0, adjust=False).mean()
    long_ema = ohlc_data['Close'].ewm(span=21, min_periods=0, adjust=False).mean()

    macd = short_ema - long_ema
    signal_line = macd.ewm(span=7, min_periods=0, adjust=False).mean()

    ohlc_data['MACD'] = macd
    ohlc_data['Signal Line'] = signal_line

    window = 20 
    num_std = 0.7 

    ohlc_data['Middle Band'] = ohlc_data['Close'].rolling(window=window).mean()
    ohlc_data['Upper Band'] = ohlc_data['Middle Band'] + (ohlc_data['Close'].rolling(window=window).std() * num_std)

    conversion_line_period = 9
    base_line_period = 26
    leading_span_b_period = 52
    displacement = 26

    tenkan_sen_high = ohlc_data['High'].rolling(window=conversion_line_period).max()
    tenkan_sen_low = ohlc_data['Low'].rolling(window=conversion_line_period).min()
    ohlc_data['Tenkan-sen'] = (tenkan_sen_high + tenkan_sen_low) / 2

    kijun_sen_high = ohlc_data['High'].rolling(window=base_line_period).max()
    kijun_sen_low = ohlc_data['Low'].rolling(window=base_line_period).min()
    ohlc_data['Kijun-sen'] = (kijun_sen_high + kijun_sen_low) / 2

    in_trade = False
    buy_price = 0
    sell_price = 0
    trade_count = 0
    total_profit = 0

    buy_dates = []
    buy_prices = []
    sell_dates = []
    sell_prices = []
    profits = []

    for index, row in ohlc_data.iterrows():
        if not in_trade:
            # Entry conditions
            if (
                row['Close'] > row['Upper Band'] and
                row['MACD'] > row['Signal Line'] and
                row['Tenkan-sen'] > row['Kijun-sen']
            ):
                in_trade = True
                buy_price = row['Close']
                buy_dates.append(index.date())
                buy_prices.append(buy_price)
                print(f"{ticker_symbol}: Buy at {index.date()} - Price: {buy_price}")
        else:
            # Exit conditions
            if not (
                row['Close'] > row['Upper Band'] and
                row['Signal Line'] > 0 and
                row['Tenkan-sen'] > row['Kijun-sen']
            ):
                sell_price = row['Close']
                profit = sell_price - buy_price
                total_profit += profit
                trade_count += 1
                sell_dates.append(index.date())
                sell_prices.append(sell_price)
                profits.append(profit)
                print(f"{ticker_symbol}: Sell at {index.date()} - Price: {sell_price} - Profit: {profit:.2f}")
                in_trade = False

    min_length = min(len(buy_dates), len(buy_prices), len(sell_dates), len(sell_prices), len(profits))

    df_trades = pd.DataFrame({
        'Buy Date': buy_dates[:min_length],
        'Buy Price': buy_prices[:min_length],
        'Sell Date': sell_dates[:min_length],
        'Sell Price': sell_prices[:min_length],
        'Profit': profits[:min_length]
    })

    df_trades.to_csv(f'{ticker_symbol}_trades_output.csv', index=False)

    if trade_count > 0:
        avg_profit_per_trade = total_profit / trade_count
        print(f"\n{ticker_symbol}: Total Trades: {trade_count}")
        print(f"{ticker_symbol}: Total Profit: {total_profit:.2f}")
        print(f"{ticker_symbol}: Average Profit per Trade: {avg_profit_per_trade:.2f}")
    else:
        print(f"{ticker_symbol}: No trades executed based on the strategy.")

ticker_symbols = ['^NSEI', '^NSEBANK', 'RELIANCE.NS', 'SBIN.NS']


start_date = '2020-01-01'
end_date = '2023-12-31'

for symbol in ticker_symbols:
    execute_trading_strategy(symbol, start_date, end_date)


[*********************100%%**********************]  1 of 1 completed
^NSEI: Buy at 2020-04-22 - Price: 9187.2998046875
^NSEI: Sell at 2020-04-23 - Price: 9313.900390625 - Profit: 126.60
^NSEI: Buy at 2020-04-24 - Price: 9154.400390625
^NSEI: Sell at 2020-04-27 - Price: 9282.2998046875 - Profit: 127.90
^NSEI: Buy at 2020-04-28 - Price: 9380.900390625
^NSEI: Sell at 2020-04-29 - Price: 9553.349609375 - Profit: 172.45
^NSEI: Buy at 2020-04-30 - Price: 9859.900390625
^NSEI: Sell at 2020-05-04 - Price: 9293.5 - Profit: -566.40
^NSEI: Buy at 2020-06-01 - Price: 9826.150390625
^NSEI: Sell at 2020-06-15 - Price: 9813.7001953125 - Profit: -12.45
^NSEI: Buy at 2020-06-22 - Price: 10311.2001953125
^NSEI: Sell at 2020-07-14 - Price: 10607.349609375 - Profit: 296.15
^NSEI: Buy at 2020-07-21 - Price: 11162.25
^NSEI: Sell at 2020-07-31 - Price: 11073.4501953125 - Profit: -88.80
^NSEI: Buy at 2020-08-26 - Price: 11549.599609375
^NSEI: Sell at 2020-08-31 - Price: 11387.5 - Profit: -162.10
^NSEI: Buy at

In [18]:
total_accuracy_rate = 0
total_trades = 0
total_winning_trades = 0
total_losing_trades = 0
total_win_loss_ratio = 0
total_profit_factor = 0

for symbol in ticker_symbols:
    
    df_trades = pd.read_csv(f'{symbol}_trades_output.csv')

    if not df_trades.empty:
        accuracy_rate = (df_trades['Profit'] > 0).sum() / len(df_trades)

        trades = len(df_trades)
        winning_trades = (df_trades['Profit'] > 0).sum()
        losing_trades = trades - winning_trades

        win_loss_ratio = winning_trades / max(1, losing_trades)
        profit_factor = abs(df_trades['Profit'][df_trades['Profit'] > 0].sum() / df_trades['Profit'][df_trades['Profit'] < 0].sum())

        print(f"\nMetrics for {symbol}:")
        print(f"Accuracy Rate: {accuracy_rate * 100:.2f}%")
        print(f"Total Trades: {trades}")
        print(f"Winning Trades: {winning_trades}")
        print(f"Losing Trades: {losing_trades}")
        print(f"Win/Loss Ratio: {win_loss_ratio:.2f}")
        print(f"Profit Factor: {profit_factor:.2f}")

        total_accuracy_rate += accuracy_rate
        total_trades += trades
        total_winning_trades += winning_trades
        total_losing_trades += losing_trades
        total_win_loss_ratio += win_loss_ratio
        total_profit_factor += profit_factor

if total_trades > 0:
    avg_accuracy_rate = total_accuracy_rate / len(ticker_symbols)
    avg_win_loss_ratio = total_win_loss_ratio / len(ticker_symbols)
    avg_profit_factor = total_profit_factor / len(ticker_symbols)

    print("\nOverall Metrics:")
    print(f"Average Accuracy Rate: {avg_accuracy_rate * 100:.2f}%")
    print(f"Total Trades Across All Symbols: {total_trades}")
    print(f"Average Win/Loss Ratio Across All Symbols: {avg_win_loss_ratio:.2f}")
    print(f"Average Profit Factor Across All Symbols: {avg_profit_factor:.2f}")
else:
    print("No trades executed based on the strategy.")



Metrics for ^NSEI:
Accuracy Rate: 52.38%
Total Trades: 42
Winning Trades: 22
Losing Trades: 20
Win/Loss Ratio: 1.10
Profit Factor: 2.36

Metrics for ^NSEBANK:
Accuracy Rate: 65.91%
Total Trades: 44
Winning Trades: 29
Losing Trades: 15
Win/Loss Ratio: 1.93
Profit Factor: 2.38

Metrics for RELIANCE.NS:
Accuracy Rate: 52.63%
Total Trades: 38
Winning Trades: 20
Losing Trades: 18
Win/Loss Ratio: 1.11
Profit Factor: 2.24

Metrics for SBIN.NS:
Accuracy Rate: 45.45%
Total Trades: 55
Winning Trades: 25
Losing Trades: 30
Win/Loss Ratio: 0.83
Profit Factor: 2.16

Overall Metrics:
Average Accuracy Rate: 54.09%
Total Trades Across All Symbols: 179
Average Win/Loss Ratio Across All Symbols: 1.24
Average Profit Factor Across All Symbols: 2.29


In [19]:
for symbol in ticker_symbols:
    df_trades = pd.read_csv(f'{symbol}_trades_output.csv')

    if not df_trades.empty:
        max_loss = df_trades['Profit'].min()
        max_gain = df_trades['Profit'].max()

        df_trades['Cumulative Profit'] = df_trades['Profit'].cumsum()
        df_trades['Previous Peak'] = df_trades['Cumulative Profit'].cummax()
        df_trades['Drawdown'] = df_trades['Cumulative Profit'] - df_trades['Previous Peak']
        max_drawdown = df_trades['Drawdown'].min()

        print(f"\nMetrics for {symbol}:")
        print(f"Max Loss: {max_loss:.2f}")
        print(f"Max Gain: {max_gain:.2f}")
        print(f"Max Drawdown: {max_drawdown:.2f}")
    else:
        print(f"{symbol}: No trades executed based on the strategy.")



Metrics for ^NSEI:
Max Loss: -566.40
Max Gain: 1441.50
Max Drawdown: -687.85

Metrics for ^NSEBANK:
Max Loss: -1338.90
Max Gain: 3765.95
Max Drawdown: -2008.15

Metrics for RELIANCE.NS:
Max Loss: -158.40
Max Gain: 458.25
Max Drawdown: -347.25

Metrics for SBIN.NS:
Max Loss: -16.70
Max Gain: 61.00
Max Drawdown: -44.65
